In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os
from matplotlib import  pyplot as plt

from mpl_toolkits import mplot3d

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
base_dir = r"/home/parth/Machine Learning/Datasets/North Corp/Ajanta"
file_list = os.listdir(base_dir)

In [3]:
file_list[0]

'MC2M7SRC0HH051022.csv'

In [4]:
filename = os.path.join(base_dir , file_list[0])
data = pd.read_csv(filename)

In [5]:
#usefull_cols = [ 'updateddate', 'total_fuel_consumption', 'total_engine_hours', 'latitude' , 'longitude',
#       'total_distance' , 'time_idle' , 'time_drive', 'speed' , 'current_fuel_level' ]
usefull_cols = [ 'updateddate', 'total_fuel_consumption', 'current_fuel_level' ]

data_use = data[usefull_cols]

In [6]:
def attributesAdder(df):
    df["dateSeconds"] = df["updateddate"].apply(
                lambda x: datetime.strptime(x , "%Y-%m-%d %H:%M:%S")).apply(
                lambda x: (x - datetime(1970,1,1)).total_seconds() )
    df["fuel_per_sec"] = df["current_fuel_level"].diff() / df["dateSeconds"].diff()
    df["total_fuel_per_sec"] = df["total_fuel_consumption"].diff() / df["dateSeconds"].diff()
    df.drop(columns="dateSeconds" , inplace=True)
    df.drop(columns="updateddate" , inplace=True)
    df = df.values
    df[np.isnan(df)] = 0
    df[np.isinf(df)] = 0
    return data_use

In [7]:
data_use = attributesAdder(data_use)

In [14]:
scaler = StandardScaler()
data_use_ = scaler.fit_transform(data_use.values)

In [16]:
pca = PCA(n_components = 0.95 )
lle = LocallyLinearEmbedding(n_components=3 , n_neighbors=10 , method='dense')
data_use_pca = pca.fit_transform(data_use_)
#data_use_lle = lle.fit_transform(data_use_)

ValueError: unrecognized method 'dense'

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=[10 , 10])
ax = plt.axes(projection="3d")

ax.scatter3D(data_use_pca[: , 0], data_use_pca[: , 1], data_use_pca[: , 2] , c=data_use_pca[: , 3] , cmap=plt.cm.brg);

plt.show()

In [ ]:
n_clusters = range(1 , 20)

means = [KMeans(n_clusters=i).fit(data_use_pca) for i in n_clusters]
scores = [mean.score(data_use_pca) for mean in means]

In [ ]:
%matplotlib inline
plt.plot(list(n_clusters) , scores)
plt.xlabel("Number of clusters")
plt.ylabel("Score")
plt.show()

In [ ]:
data["cluster"] = means[17].predict(data_use_pca)

In [17]:
markcluster = data["cluster"].value_counts() < 100
markcluster = dict(markcluster)

data["Anomaly"] = None
for index in range(len(data)):
    data.at[index , "Anomaly"] = markcluster[data["cluster"].iloc[ index ]]
    
data.to_csv("HD anomaly {}.csv".format(filename.split(' ')[0]))

KeyError: 'cluster'

In [ ]:
data["Anomaly"]